# 基于MindSpore的GPT2文本摘要

## 环境配置

1. 配置python3.9环境


In [2]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [ ]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

***注：以上代码执行完成后，需点击左上角或右上角将kernel更换为python-3.9.0***

2. 安装mindspore2.2.12，安装指南详见：[MindSpore安装](https://www.mindspore.cn/install)
3. 安装MindNLP及相关依赖，MindNLP官方仓详见：[MindNLP](https://github.com/mindspore-lab/mindnlp)

In [21]:
!pip install mindnlp-0.4.1-py3-none-any.whl  # 将安装mindnlp版本更换为mindnlp-0.4.0-py3-none-any.whl（daily版本）
!pip install tokenizers==0.19.1 -i https://pypi.tuna.tsinghua.edu.cn/simple  # 修改tokenizers版本为0.19.1
%env no_proxy='a.test.com,127.0.0.1,2.2.2.2'
%pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.4.0/MindSpore/unified/aarch64/mindspore-2.4.0-cp39-cp39-linux_aarch64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip uninstall mindformers -y

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
Processing ./mindnlp-0.4.1-py3-none-any.whl
mindnlp is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.
env: no_proxy='a.test.com,127.0.0.1,2.2.2.2'
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.4.0/MindSpore/unified/aarch64/mindspore-2.4.0-cp39-cp39-linux_aarch64.whl (333.7 MB)
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart

***注：执行如上命令完成安装后，请点击上方的restart kernel图标重启kernel，再进行实验***

### 数据集加载与处理

1. 数据集加载

    本次实验使用的是nlpcc2017摘要数据，内容为新闻正文及其摘要，总计50000个样本。

In [1]:
from mindnlp.utils import http_get

# download dataset
url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
path = http_get(url, './')

[WARNING] GE_ADPT(12319,ffffaba360b0,python):2024-12-03-21:03:05.821.904 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleGetModelId failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleGetModelId
[WARNING] GE_ADPT(12319,ffffaba360b0,python):2024-12-03-21:03:05.821.956 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleLoadFromMem failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleLoadFromMem
[WARNING] GE_ADPT(12319,ffffaba360b0,python):2024-12-03-21:03:05.821.975 [mindspore/ccsrc/utils/dlopen_macro.h:163] DlsymAscend] Dynamically load symbol aclmdlBundleUnload failed, result = /usr/local/Ascend/ascend-toolkit/latest/lib64/libascendcl.so: undefined symbol: aclmdlBundleUnload
[WARNING] GE_ADPT(12319,ffffaba360b0,python):2024-12-03-21:03:05.822.162 [mindspore/ccsrc/utils/dlopen_macro.h:163] Dl

In [2]:
from mindspore.dataset import TextFileDataset

# load dataset
dataset = TextFileDataset(str(path), shuffle=False)
dataset.get_dataset_size()

50000

**本案例默认在GPU P100上运行，因中文文本，tokenizer使用的是bert tokenizer而非gpt tokenizer等原因，全量数据训练1个epoch的时间约为80分钟。**

**为节约时间，我们选取了数据集中很小的一个子集（500条数据）来演示gpt2的微调和推理全流程，但由于数据量不足，会导致模型效果较差。**

In [3]:
# split into training and testing dataset
mini_dataset, _ = dataset.split([0.001, 0.999], randomize=False)
train_dataset, test_dataset = mini_dataset.split([0.9, 0.1], randomize=False)

2. 数据预处理

    原始数据格式：
    ```text
    article: [CLS] article_context [SEP]
    summary: [CLS] summary_context [SEP]
    ```
    预处理后的数据格式：

    ```text
    [CLS] article_context [SEP] summary_context [SEP]
    ```

In [4]:
import json
import numpy as np

# preprocess dataset
def process_dataset(dataset, tokenizer, batch_size=4, max_seq_len=1024, shuffle=False):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def merge_and_pad(article, summary):
        # tokenization
        # pad to max_seq_length, only truncate the article
        tokenized = tokenizer(text=article, text_pair=summary,
                              padding='max_length', truncation='only_first', max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['input_ids']
    
    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    # change column names to input_ids and labels for the following training
    dataset = dataset.map(merge_and_pad, ['article', 'summary'], ['input_ids', 'labels'])

    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(batch_size)

    return dataset

因GPT2无中文的tokenizer，我们使用BertTokenizer替代。

In [5]:
from mindnlp.transformers import BertTokenizer

# We use BertTokenizer for tokenizing chinese context.
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
len(tokenizer)

/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted, and will be then set to `False` by default. 
  warnings.warn(


21128

In [6]:
train_dataset = process_dataset(train_dataset, tokenizer, batch_size=1)

In [7]:
next(train_dataset.create_tuple_iterator())

[Tensor(shape=[1, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0]]),
 Tensor(shape=[1, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0]])]

### 模型构建

1. 构建GPT2ForSummarization模型，注意***shift right***的操作。

In [8]:
# 修改部分代码
# from mindspore import ops
# from mindnlp.transformers import GPT2LMHeadModel

# class GPT2ForSummarization(GPT2LMHeadModel):
#     def construct(
#         self,
#         input_ids = None,
#         attention_mask = None,
#         labels = None,
#     ):
#         outputs = super().construct(input_ids=input_ids, attention_mask=attention_mask)
#         shift_logits = outputs.logits[..., :-1, :]
#         shift_labels = labels[..., 1:]
#         # Flatten the tokens
#         loss = ops.cross_entropy(shift_logits.view(-1, shift_logits.shape[-1]), shift_labels.view(-1), ignore_index=tokenizer.pad_token_id)
#         return loss
from mindnlp.core.nn import functional as F
from mindnlp.transformers import GPT2LMHeadModel

class GPT2ForSummarization(GPT2LMHeadModel):
    def forward(
        self,
        input_ids = None,
        attention_mask = None,
        labels = None,
    ):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask)
        shift_logits = outputs.logits[..., :-1, :]
        shift_labels = labels[..., 1:]
        # Flatten the tokens
        loss = F.cross_entropy(shift_logits.view(-1, shift_logits.shape[-1]), shift_labels.view(-1), ignore_index=tokenizer.pad_token_id)
        return (loss,)

2. 动态学习率

In [9]:
# 去除了动态学习率

# from mindspore import ops
# from mindspore.nn.learning_rate_schedule import LearningRateSchedule

# class LinearWithWarmUp(LearningRateSchedule):
#     """
#     Warmup-decay learning rate.
#     """
#     def __init__(self, learning_rate, num_warmup_steps, num_training_steps):
#         super().__init__()
#         self.learning_rate = learning_rate
#         self.num_warmup_steps = num_warmup_steps
#         self.num_training_steps = num_training_steps

#     def construct(self, global_step):
#         if global_step < self.num_warmup_steps:
#             return global_step / float(max(1, self.num_warmup_steps)) * self.learning_rate
#         return ops.maximum(
#             0.0, (self.num_training_steps - global_step) / (max(1, self.num_training_steps - self.num_warmup_steps))
#         ) * self.learning_rate

### 模型训练

In [10]:
num_epochs = 1
warmup_steps = 100
learning_rate = 1.5e-4
max_grad_norm = 1.0
num_training_steps = num_epochs * train_dataset.get_dataset_size()

In [11]:
from mindspore import nn
from mindnlp.transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(vocab_size=len(tokenizer))
model = GPT2ForSummarization(config)
# 修改部分代码
# lr_scheduler = LinearWithWarmUp(learning_rate=learning_rate, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)
# optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=lr_scheduler)
# optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=learning_rate)

GPT2LMHeadModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
[WARNING] DEVICE(12319,ffffaba360b0,python):2024-12-03-21:03:34.197.849 [mindspore/ccsrc/plugin/device/ascend/hal/device/ascend_vmm_adapter.h:188] CheckVmmDriverVersion] Driver version is less than 24.0.0, vmm is disabled by default, drvier_version: 23.0.6


In [12]:
# 记录模型参数数量
print('number of model parameters: {}'.format(model.num_parameters()))

number of model parameters: 102068736


In [13]:
# 修改部分代码
# from mindnlp._legacy.engine import Trainer
# from mindnlp._legacy.engine.callbacks import CheckpointCallback

# ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt2_summarization',
#                                 epochs=1, keep_checkpoint_max=2)

# trainer = Trainer(network=model, train_dataset=train_dataset,
#                   epochs=num_epochs, optimizer=optimizer, callbacks=ckpoint_cb)
# trainer.set_amp(level='O1')  # 开启混合精度

In [14]:
from mindnlp.engine import TrainingArguments

training_args = TrainingArguments(
    output_dir="gpt2_summarization",
    save_steps=train_dataset.get_dataset_size(),
    save_total_limit=3,
    logging_steps=1000,
    max_steps=num_training_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    warmup_steps=warmup_steps
    
)

from mindnlp.engine import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [15]:
# 修改部分代码
# trainer.run(tgt_columns="labels")
trainer.train()

  0%|          | 0/45 [00:00<?, ?it/s]

100%|██████████| 45/45 [00:32<00:00,  1.38it/s]

{'train_runtime': 32.5297, 'train_samples_per_second': 11.067, 'train_steps_per_second': 1.383, 'train_loss': 9.11246066623264, 'epoch': 1.0}


TrainOutput(global_step=45, training_loss=9.11246066623264, metrics={'train_runtime': 32.5297, 'train_samples_per_second': 11.067, 'train_steps_per_second': 1.383, 'train_loss': 9.11246066623264, 'epoch': 1.0})

In [16]:
def process_test_dataset(dataset, tokenizer, batch_size=1, max_seq_len=1024, max_summary_len=100):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def pad(article):
        tokenized = tokenizer(text=article, truncation=True, max_length=max_seq_len-max_summary_len)
        return tokenized['input_ids']

    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(pad, 'article', ['input_ids'])
    
    dataset = dataset.batch(batch_size)

    return dataset

In [17]:
batched_test_dataset = process_test_dataset(test_dataset, tokenizer, batch_size=1)

In [18]:
print(next(batched_test_dataset.create_tuple_iterator(output_numpy=True)))

[array([[ 101, 4373, 3360, 3173, 7319, 5381,  118, 4373, 3360, 3241, 2845,
        6380, 8020, 6858, 6380, 1447,  133,  100,  135, 7942,  837, 2412,
        8021, 5298, 6756, 7313, 7395, 8024, 1355, 4385, 6378, 5298, 1767,
        7353, 6818,  671, 3418, 4510, 5296, 3327,  677, 3300,  671,  702,
        7881, 4973, 8024, 8128, 2259, 4638, 1144, 3378, 4989, 1315, 4260,
         677, 1343, 2929, 7881, 8024,  679, 2682, 2218, 3634, 1462,  700,
        7942, 3787,  511, 3189, 1184, 1355, 4495, 1762, 1300, 4635, 1344,
        3152, 1765, 7252,  671, 7730, 3413, 6378, 5298, 1767, 4638, 2692,
        1912,  752, 3125, 8024,  808,  782, 1537, 1656,  511,  791, 2399,
        8128, 2259, 4638, 1144, 3378, 3221, 1300, 4635, 1344, 2123, 4059,
        7252, 3173, 5783, 3333,  782, 8024,  679,  719, 1184, 1168, 1071,
        1828, 1356, 1144, 3378, 3378,  818, 3136, 5298, 4638, 2123, 4059,
        7252, 3378, 7730, 3413, 1346, 1217, 3749, 6756, 7730, 7724, 1447,
        1824, 6378,  511, 8132, 3189,

In [19]:
# 修改部分代码
# model = GPT2LMHeadModel.from_pretrained('./checkpoint/gpt2_summarization_epoch_0.ckpt', config=config)
model = GPT2LMHeadModel.from_pretrained('./gpt2_summarization/checkpoint-45', config=config)

由于训练数据量少，epochs数少且tokenizer并未使用gpt tokenizer等因素，模型推理效果会较差。

In [20]:
model.set_train(False)
model.config.eos_token_id = model.config.sep_token_id
i = 0
for (input_ids, raw_summary) in batched_test_dataset.create_tuple_iterator():
    output_ids = model.generate(input_ids, max_new_tokens=50, num_beams=5, no_repeat_ngram_size=2)
    output_text = tokenizer.decode(output_ids[0].tolist())
    print(output_text)
    i += 1
    if i == 1:
        break

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


[CLS] 玉 林 新 闻 网 - 玉 林 晚 报 讯 （ 通 讯 员 < [UNK] > 黄 传 庆 ） 练 车 间 隙 ， 发 现 训 练 场 附 近 一 根 电 线 杆 上 有 一 个 鸟 窝 ， 21 岁 的 刁 某 立 即 爬 上 去 捉 鸟 ， 不 想 就 此 命 丧 黄 泉 。 日 前 发 生 在 博 白 县 文 地 镇 一 驾 校 训 练 场 的 意 外 事 故 ， 令 人 唏 嘘 。 今 年 21 岁 的 刁 某 是 博 白 县 宁 潭 镇 新 荣 村 人 ， 不 久 前 到 其 堂 叔 刁 某 某 任 教 练 的 宁 潭 镇 某 驾 校 参 加 汽 车 驾 驶 员 培 训 。 25 日 中 午 ， 刁 某 随 堂 叔 刁 某 某 及 另 外 几 名 学 员 开 车 到 文 地 镇 ， 借 用 文 地 镇 钛 白 粉 厂 附 近 的 训 练 场 练 车 。 14 时 许 ， 训 练 间 隙 ， 刁 某 发 现 训 练 场 附 近 一 根 电 线 杆 上 有 一 个 鸟 窝 ， 一 只 [UNK] 八 哥 [UNK] 正 叼 着 虫 子 飞 回 鸟 窝 。 平 时 刁 某 偶 尔 会 捕 鸟 出 卖 ， 知 道 这 种 鸟 价 值 100 多 元 。 他 跟 身 边 学 员 打 了 个 招 呼 ， 便 飞 奔 过 去 ， 欲 爬 上 电 线 杆 捉 鸟 。 刁 某 某 发 现 后 追 过 去 欲 阻 止 ， 但 为 时 已 晚 ， 等 刁 某 某 赶 到 时 ， 刁 某 已 爬 到 电 线 杆 顶 ， 伸 手 捉 鸟 时 不 慎 触 碰 到 头 顶 的 高 压 电 线 ， 当 即 身 亡 。 其 手 掌 被 外 露 的 钢 枝 刺 穿 ， 尸 体 悬 挂 在 电 线 杆 上 。 26 日 下 午 ， 在 文 地 、 宁 潭 镇 政 府 及 相 关 部 门 协 调 下 ， 相 关 责 任 方 与 死 者 家 属 达 成 赔 偿 协 议 ， 死 者 家 属 同 意 将 死 者 尸 体 取 下 搬 走 ， 相 关 责 任 方 共 赔 偿 死 者 家 属 15. 8 万 元 ， 其 中 宁 潭 镇 某 驾 校 赔 付 7. 3 万 元 ， 死 者 堂 叔 刁 某 某 （ 驾 校 教 练 ） 赔 付 5. 3 万 元 ， 文 地 供 电 所 本 来 没 有 直